In [444]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException

import getpass

from bs4 import BeautifulSoup, Comment
import re
import pandas as pd
import os
import numpy as np


In [445]:
url ="https://www.tradingview.com"
driver = webdriver.Firefox()

In [446]:
driver.get(url)

In [447]:
driver.find_elements_by_xpath("//a[contains(text(),'Sign In')]")[1].click()

In [448]:
driver.find_element_by_name('username').send_keys('bame4')
driver.find_element_by_name('password').send_keys('quantorithm123')

In [449]:
driver.find_element_by_tag_name('body').send_keys(Keys.ENTER)


In [450]:
def scrape(cname):
    info = soup.find_all( "div", class_="tv-site-widget tv-widget-idea js-widget-idea")
    df = pd.DataFrame(columns=['Time Stamp', 'Username', 'Topic', 'Text', 'Direction', 'Views', 'Cmt.Num', 'Like'])

    for i in range(1, pagenum+1):
        url = "https://www.tradingview.com/symbols/{}/page-{}/".format(cname, i)
        driver.get(url)

        for data in info:
            user = data.find('img')['alt']
            topic = data.find(class_="tv-widget-idea__title-name apply-overflow-tooltip").get_text()
            link = 'https://tradingview.com' + data.find(class_='js-widget-idea__popup')['data-href-idea-custom-link']
            views = data.find(class_="tv-social-stats__count").get_text()
            time = data.find(class_="tv-widget-idea__time")['data-timestamp']

            like = data.find_all(class_="tv-social-stats__count")
            com = like[1].get_text()
            lik =like[2].get_text()
            direction = data.find(class_="tv-idea-label tv-idea-label--long i-except-phones-only")

            try:
                direction = direction.get_text()
            except:
                try:
                    direction = data.find(class_="tv-idea-label tv-idea-label--short i-except-phones-only")
                    direction = direction.get_text()
                except:
                    pass

            dec = 'https://tradingview.com' + data.find(class_='js-widget-idea__popup')['data-href-idea-custom-link']
            driver.get(dec)
            soup1=BeautifulSoup(driver.page_source, 'lxml')
            
            text = soup1.find(class_="tv-chart-view__description-wrap js-chart-view__description").get_text()
            
            df = df.append({'Time Stamp':time, 'Username':user, 'Topic':topic, 'Text':text, 'Views':views, 'Direction':direction, 'Cmt.Num':com, 'Like':lik}, ignore_index=True)      
            df.to_csv('temp.csv')
            driver.back()
            
        return df

In [451]:
coinname = {'BTCUSD', 'LTCUSD', 'ETHUSD', 'DASHUSD', 'STRUSD', 'DOGEUSD', 'XRPUSD', 'XMRUSD'}


for key in coinname:
    driver.get("https://www.tradingview.com/symbols/{}/".format(key))
    #Selenium hands the page source to Beautiful Soup
    soup=BeautifulSoup(driver.page_source, 'lxml')

    page = soup.find(class_="tv-load-more__pagination js-feed-pagination")

    lastp = soup.find_all(class_="tv-load-more__page")

    pagenum = int(lastp[-1].get_text())
    df = scrape(key)
    df.to_csv("{}.csv".format(key))

KeyboardInterrupt: 